In [297]:
# -*- coding: UTF-8 -*-
import pandas as pd
from pandas.io.json import json_normalize
import json
from collections import Counter


In [298]:
filepath = '/Users/alexandraplassaras/src/spark_joy/comments.json'

In [299]:
with open(filepath, 'r') as f:
    data = json.load(f)

In [300]:
comment_list = []
for item in data['items']:
    comment_row = item['snippet']['topLevelComment']['snippet']
    del comment_row['authorDisplayName']
    del comment_row['authorProfileImageUrl']
    del comment_row['canRate']
    del comment_row['updatedAt']
    del comment_row['viewerRating']
    del comment_row['textOriginal']
    del comment_row['authorChannelUrl']
    del comment_row['likeCount']

    comment_list.append(comment_row)

#     snippet = data['items'][0]['snippet']['topLevelComment']['snippet']

In [301]:
df = pd.DataFrame(comment_list)

In [302]:
# df.head()

In [303]:
def extract_emojis(sentence):
    return [word for word in sentence.split() if str(word.encode('unicode-escape'))[2] == '\\' ]


In [304]:
df['emoji'] = df['textDisplay'].apply(lambda x: extract_emojis(x))

In [305]:
# df.head()

In [306]:
emoji_lst = df['emoji'].tolist()

In [307]:
main_emoji = []
for i in emoji_lst:
    if i == []:
        continue
    for j in i:
        for k in j:
            main_emoji.append(k)

In [308]:
counts = Counter(main_emoji)
print(counts)
emoji_df = pd.DataFrame.from_dict(counts, orient='index').reset_index()
emoji_df = emoji_df.rename(index=str, columns={"index": "emoji", 0: "count"})

emoji_df = emoji_df.sort_values(by=['count'], ascending = False)
emoji_df.head()

Counter({'️': 17, '💕': 15, '❤': 11, '😍': 9, '😊': 7, '👍': 7, '👏': 6, '💖': 6, '\U0001f929': 4, '☺': 4, '😂': 4, '🙂': 4, '🌸': 4, '💜': 3, '😉': 3, '😘': 3, '\U0001f970': 2, '👋': 2, '💟': 2, '🏼': 2, '😩': 1, '🌹': 1, '💐': 1, '💘': 1, '🤣': 1, '🙌': 1, '🏽': 1, '😃': 1, '😆': 1, '.': 1, '😰': 1, '💗': 1, '✨': 1, '😁': 1, '“': 1, 't': 1, 'h': 1, 'e': 1, '😋': 1, '🤗': 1, '👌': 1, '🙏': 1, '💋': 1, '😚': 1, '♥': 1, '‼': 1, '😄': 1, '🌞': 1})


,emoji,count
5,️,17
3,💕,15
4,❤,11
1,😍,9
12,👍,7


In [309]:
emoji_df['pct'] = round((emoji_df['count']/emoji_df['count'].sum()) * 100, 2)
emoji_df.head()

,emoji,count,pct
5,️,17,11.89
3,💕,15,10.49
4,❤,11,7.69
1,😍,9,6.29
12,👍,7,4.90
